In [9]:
from pyspark.sql.functions import *
trainDF = spark.read.parquet('/data/user/hive/warehouse/ian/feature/testDF')

trainDF = trainDF.withColumn('m1',substring(trainDF.mac,1,1)).\
      withColumn('m2',substring(trainDF.mac,2,1)).\
      withColumn('m3',substring(trainDF.mac,3,1)).\
      withColumn('m4',substring(trainDF.mac,4,1)).\
      withColumn('m5',substring(trainDF.mac,5,1)).\
      withColumn('m6',substring(trainDF.mac,6,1)).\
      withColumn('m7',substring(trainDF.mac,7,1)).\
      withColumn('m8',substring(trainDF.mac,8,1)).\
      withColumn('m9',substring(trainDF.mac,9,1)).\
      withColumn('m10',substring(trainDF.mac,10,1)).\
      withColumn('m11',substring(trainDF.mac,11,1)).\
      withColumn('m12',substring(trainDF.mac,12,1))

df = trainDF.withColumn('f1',conv(trainDF.m1, 16, 10)).withColumn('f2',conv(trainDF.m2, 16, 10)).\
        withColumn('f3',conv(trainDF.m3, 16, 10)).withColumn('f4',conv(trainDF.m4, 16, 10)).\
        withColumn('f5',conv(trainDF.m5, 16, 10)).withColumn('f6',conv(trainDF.m6, 16, 10)).\
        withColumn('f7',conv(trainDF.m7, 16, 10)).withColumn('f8',conv(trainDF.m8, 16, 10)).\
        withColumn('f9',conv(trainDF.m9, 16, 10)).withColumn('f10',conv(trainDF.m10, 16, 10)).\
        withColumn('f11',conv(trainDF.m11, 16, 10)).withColumn('f12',conv(trainDF.m12, 16, 10))

df = df.select('mac','id',col('f1').cast('float'),col('f2').cast('float')\
               ,col('f3').cast('float'),col('f4').cast('float')\
               ,col('f5').cast('float'),col('f6').cast('float')\
               ,col('f7').cast('float'),col('f8').cast('float')\
               ,col('f9').cast('float'),col('f10').cast('float')\
               ,col('f11').cast('float'),col('f12').cast('float'))

from pyspark.ml.feature import VectorAssembler
vec = VectorAssembler(inputCols=['f1','f2','f3','f4','f5','f6','f7','f8','f9','f10','f11','f12'],outputCol='features')
new_df = vec.transform(df)

from pyspark.ml.feature import StringIndexer
si = StringIndexer(inputCol='id', outputCol='label')
si_model = si.fit(new_df)
df_final = si_model.transform(new_df)

df_final.show(5,truncate=False)

+------------+-----+---+---+---+----+----+----+----+----+---+----+----+---+-------------------------------------------------------+-----+
|mac         |id   |f1 |f2 |f3 |f4  |f5  |f6  |f7  |f8  |f9 |f10 |f11 |f12|features                                               |label|
+------------+-----+---+---+---+----+----+----+----+----+---+----+----+---+-------------------------------------------------------+-----+
|0008221E6CF5|39226|0.0|0.0|0.0|8.0 |2.0 |2.0 |1.0 |14.0|6.0|12.0|15.0|5.0|[0.0,0.0,0.0,8.0,2.0,2.0,1.0,14.0,6.0,12.0,15.0,5.0]   |1.0  |
|000EC6DC7BE2|49814|0.0|0.0|0.0|14.0|12.0|6.0 |13.0|12.0|7.0|11.0|14.0|2.0|[0.0,0.0,0.0,14.0,12.0,6.0,13.0,12.0,7.0,11.0,14.0,2.0]|2.0  |
|001636154AF9|49813|0.0|0.0|1.0|6.0 |3.0 |6.0 |1.0 |5.0 |4.0|10.0|15.0|9.0|[0.0,0.0,1.0,6.0,3.0,6.0,1.0,5.0,4.0,10.0,15.0,9.0]    |5.0  |
|0016785B0DF5|49810|0.0|0.0|1.0|6.0 |7.0 |8.0 |5.0 |11.0|0.0|13.0|15.0|5.0|[0.0,0.0,1.0,6.0,7.0,8.0,5.0,11.0,0.0,13.0,15.0,5.0]   |3.0  |
|0019CBF145D0|49807|0.0|0.0|1.0|9.

In [10]:
df_final.write.mode('overwrite').parquet('hdfs:///data/user/hive/warehouse/ian/feature/testDF_mac_12',compression='gzip')

# Training with non cv

In [4]:
trainDF = spark.read.parquet('/data/user/hive/warehouse/ian/feature/trainDF_apple')
testDF = spark.read.parquet('/data/user/hive/warehouse/ian/feature/testDF_apple')

In [5]:
trainDF.count(),testDF.count()

(7288049, 3121586)

In [6]:
#from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [7]:
rf = RandomForestClassifier(labelCol='label',featuresCol='features',numTrees=10)

In [8]:
model = rf.fit(trainDF)

In [9]:
predictions = model.transform(testDF)

In [10]:
predictions.dtypes

[('mac', 'string'),
 ('id', 'string'),
 ('cn_name', 'string'),
 ('f1', 'float'),
 ('f2', 'float'),
 ('f3', 'float'),
 ('f4', 'float'),
 ('f5', 'float'),
 ('f6', 'float'),
 ('f7', 'float'),
 ('f8', 'float'),
 ('f9', 'float'),
 ('f10', 'float'),
 ('f11', 'float'),
 ('f12', 'float'),
 ('features', 'vector'),
 ('label', 'double'),
 ('rawPrediction', 'vector'),
 ('probability', 'vector'),
 ('prediction', 'double')]

In [11]:
predictions.select('label','prediction','mac').show(5,truncate=False)

+-----+----------+------------+
|label|prediction|mac         |
+-----+----------+------------+
|5.0  |5.0       |000726429552|
|1.0  |1.0       |0007E93000C0|
|8.0  |5.0       |00082284245B|
|1.0  |5.0       |000B2F545632|
|1.0  |1.0       |000CE74FC714|
+-----+----------+------------+
only showing top 5 rows



In [12]:
evaluator = MulticlassClassificationEvaluator(labelCol='label',predictionCol='prediction',metricName='accuracy')
accuracy = evaluator.evaluate(predictions)

In [13]:
accuracy

0.4759468424063921

# Training with CV & hp tuning Random Forest

In [ ]:
trainDF = spark.read.parquet('/data/user/hive/warehouse/ian/feature/trainDF_apple')
testDF = spark.read.parquet('/data/user/hive/warehouse/ian/feature/testDF_apple')

from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator,ParamGridBuilder
import numpy as np

rf = RandomForestClassifier()

paramGrid_rf = ParamGridBuilder()\
       .addGrid(rf.maxDepth,[3,4,5,6,7,8,9,10])\
       .addGrid(rf.numTrees,[10,15,20,25,30])\
       .build()

crossval_rf = CrossValidator(estimator=rf,
                            estimatorParamMaps=paramGrid_rf,
                            evaluator=MulticlassClassificationEvaluator(),
                            numFolds=5)

cvModel_rf = crossval_rf.fit(trainDF)

best_model_rf = cvModel_rf.bestModel
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
my_eval_rf = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='accuracy')
my_eval_rf.evaluate(best_model_rf.transform(testDF))

In [4]:
#best_model_rf.save('hdfs:///data/user/hive/warehouse/ian/model/mac_id_12_rf_cv_tuning_model')

In [4]:
result = best_model_rf.transform(testDF)

In [10]:
result.select('id','label','prediction').show(truncate=False)

+-----+-----+----------+
|id   |label|prediction|
+-----+-----+----------+
|39226|1.0  |1.0       |
|49814|2.0  |1.0       |
|49813|5.0  |1.0       |
|49810|3.0  |1.0       |
|49807|0.0  |1.0       |
|49808|6.0  |10.0      |
|10679|15.0 |1.0       |
|49807|0.0  |0.0       |
|49807|0.0  |0.0       |
|49807|0.0  |0.0       |
|49807|0.0  |0.0       |
|10680|10.0 |10.0      |
|10680|10.0 |10.0      |
|10680|10.0 |10.0      |
|10680|10.0 |10.0      |
|39147|7.0  |0.0       |
|39226|1.0  |0.0       |
|39000|8.0  |0.0       |
|49807|0.0  |0.0       |
|49807|0.0  |0.0       |
+-----+-----+----------+
only showing top 20 rows



In [11]:
mapping_table = result.select('id','label').distinct()

In [10]:
#mapping_table.write.mode('overwrite').parquet('hdfs:///data/user/hive/warehouse/ian/feature/mapping_table',compression='gzip')

In [12]:
mapping_table = mapping_table.withColumnRenamed('id','predict_id')

In [13]:
result = result.select('mac','prediction')
final_result = result.join(mapping_table,result.prediction == mapping_table.label)

In [14]:
final_result = final_result.select('mac','predict_id').distinct()

In [17]:
final_result.sample(0.0001).show(truncate=False)

+------------+----------+
|mac         |predict_id|
+------------+----------+
|8C8590DA1CCB|49813     |
|DCA4CA216A52|49814     |
|00B362D13FDD|49809     |
|38CADA970EF3|49809     |
|B844D9DBFB9F|49808     |
|5433CB5B8F9E|39226     |
|649ABE637666|49807     |
|7014A6CA6D7C|49807     |
|D88F76C7CFEF|39226     |
|24F6773B8DBB|49814     |
|844167A3A34C|49807     |
|40261961E073|39147     |
|6C4D73481425|39147     |
|483B3895CD20|49809     |
|50A67FA7A883|49880     |
|68EF43BF12BA|39147     |
|48A195707F1E|49814     |
|DC2B2A2A969A|49809     |
|40831DA2E1CE|49813     |
|C0CCF84C4B98|49810     |
+------------+----------+
only showing top 20 rows



In [104]:
testDF.filter(testDF.mac == '8C8590DA1CCB').show()

+------------+-----+---+----+---+---+---+---+----+----+---+----+----+----+--------------------+-----+
|         mac|   id| f1|  f2| f3| f4| f5| f6|  f7|  f8| f9| f10| f11| f12|            features|label|
+------------+-----+---+----+---+---+---+---+----+----+---+----+----+----+--------------------+-----+
|8C8590DA1CCB|49814|8.0|12.0|8.0|5.0|9.0|0.0|13.0|10.0|1.0|12.0|12.0|11.0|[8.0,12.0,8.0,5.0...|  2.0|
+------------+-----+---+----+---+---+---+---+----+----+---+----+----+----+--------------------+-----+



# crontab mac_id prediction

In [21]:
from pyspark.ml.classification import RandomForestClassificationModel
model = RandomForestClassificationModel.load('/data/user/hive/warehouse/ian/model/mac_id_12_rf_cv_tuning_model')
testDF = spark.read.parquet('/data/user/hive/warehouse/ian/feature/testDF_mac_12') #change PATH
result = model.transform(testDF)
mapping_table = spark.read.parquet('/data/user/hive/warehouse/ian/feature/mapping_table').withColumnRenamed('id','predict_id')
result = result.select('mac','prediction')
final_result = result.join(mapping_table,result.prediction == mapping_table.label)
final_result = final_result.select('mac','predict_id').distinct()
final_result.sample(0.0001).show(truncate=False)

+------------+----------+
|mac         |predict_id|
+------------+----------+
|ECADB8719CA6|49866     |
|B8C111A99F24|39226     |
|649ABE71B5D0|49807     |
|8489AD949EC9|49807     |
|D461DA1A2825|39147     |
|4C32757E1FA0|49866     |
|00CDFE530E6F|49809     |
|ECADB806B025|49880     |
|2078F0AD2E87|49880     |
|F8279307EF41|49806     |
|E49ADC79CA71|39226     |
|F0DBE2AAAEDA|49807     |
|D81D72C96C00|49808     |
|703C6947FE47|10950     |
|0CD7460C8518|49810     |
|58E6BAA3071F|39890     |
|5CF7E69B2E63|49809     |
|E49A793DAD52|49809     |
|C83C85CD9318|39226     |
|BCA920CE4956|49814     |
+------------+----------+
only showing top 20 rows



In [31]:
testDF.filter(testDF.mac == '649ABE71B5D0').show()

+------------+-----+---+---+---+----+----+----+---+---+----+---+----+---+--------------------+-----+
|         mac|   id| f1| f2| f3|  f4|  f5|  f6| f7| f8|  f9|f10| f11|f12|            features|label|
+------------+-----+---+---+---+----+----+----+---+---+----+---+----+---+--------------------+-----+
|649ABE71B5D0|49807|6.0|4.0|9.0|10.0|11.0|14.0|7.0|1.0|11.0|5.0|13.0|0.0|[6.0,4.0,9.0,10.0...|  0.0|
+------------+-----+---+---+---+----+----+----+---+---+----+---+----+---+--------------------+-----+



# apple_mac_prediction

In [2]:
from pyspark.ml.classification import RandomForestClassificationModel
from pyspark.sql.functions import *
model = RandomForestClassificationModel.load('/data/user/hive/warehouse/ian/model/mac_id_12_rf_cv_tuning_model')

d1 = spark.read.csv('/user/maxnet/database/sig.db/data_visual_macinfo_res',sep='\x01').select('_c0','_c4','_c5').distinct().withColumnRenamed('_c0','mac').withColumnRenamed('_c4','manu').withColumnRenamed('_c5','prior')
d2 = d1.filter(d1.prior < 4)
d3 = d2.filter(d2.manu == '苹果').select('mac')

df = d3.withColumn('m1',substring(d3.mac,1,1))\
       .withColumn('m2',substring(d3.mac,2,1))\
       .withColumn('m3',substring(d3.mac,3,1))\
       .withColumn('m4',substring(d3.mac,4,1))\
       .withColumn('m5',substring(d3.mac,5,1))\
       .withColumn('m6',substring(d3.mac,6,1))\
       .withColumn('m7',substring(d3.mac,7,1))\
       .withColumn('m8',substring(d3.mac,8,1))\
       .withColumn('m9',substring(d3.mac,9,1))\
       .withColumn('m10',substring(d3.mac,10,1))\
       .withColumn('m11',substring(d3.mac,11,1))\
       .withColumn('m12',substring(d3.mac,12,1))  

df = df.withColumn('f1',conv(df.m1, 16, 10)).withColumn('f2',conv(df.m2, 16, 10))\
                         .withColumn('f3',conv(df.m3, 16, 10)).withColumn('f4',conv(df.m4, 16, 10))\
                         .withColumn('f5',conv(df.m5, 16, 10)).withColumn('f6',conv(df.m6, 16, 10))\
                         .withColumn('f7',conv(df.m7, 16, 10)).withColumn('f8',conv(df.m8, 16, 10))\
                         .withColumn('f9',conv(df.m9, 16, 10)).withColumn('f10',conv(df.m10, 16, 10))\
                         .withColumn('f11',conv(df.m11, 16, 10)).withColumn('f12',conv(df.m12, 16, 10))

df = df.select('mac',\
                col('f1').cast('float'),col('f2').cast('float'),\
                col('f3').cast('float'),col('f4').cast('float'),\
                col('f5').cast('float'),col('f6').cast('float'),\
                col('f7').cast('float'),col('f8').cast('float'),\
                col('f9').cast('float'),col('f10').cast('float'),\
                col('f11').cast('float'),col('f12').cast('float'))

from pyspark.ml.feature import VectorAssembler
vec = VectorAssembler(inputCols=['f1','f2','f3','f4','f5','f6','f7','f8','f9','f10','f11','f12'],outputCol='features')
df = vec.transform(df)

result = model.transform(df)
mapping_table = spark.read.parquet('/data/user/hive/warehouse/ian/feature/mapping_table').withColumnRenamed('id','predict_id')
result = result.select('mac','prediction')
tmp = result.join(mapping_table,result.prediction == mapping_table.label)
prediction = tmp.select('mac','predict_id').distinct()

prediction.write.saveAsTable("sig.apple_mac_prediction", None, "overwrite", None)